In [17]:
from dotenv import load_dotenv
from pathlib import Path
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)

import os
API_KEY_MAPS = os.getenv("API_KEY_MAPS")

In [18]:
from pymongo import MongoClient

client = MongoClient()  
db = client['webscraping'] # access db
collection = db['documents'] # access collection

In [19]:
def search_topics_with_address(document):
    topics = []

    for pauta in document.get('pautas', []):
        if (pauta['assunto'].lower().find(' rua ') > 0) or (pauta['assunto'].lower().find(' avenida ') > 0) or (pauta['assunto'].lower().find(' bairro ') > 0):
            topics.append(pauta['assunto'])

    flag = False
    address = ""
    adresses = []
    count = 1
    for topic in topics:
        for word in topic.split(' '):
            if word.lower() in ['rua', 'avenida', 'bairro'] or flag:
                flag = True
                address += word + ' '
                count += 1
            if count == 5:
                flag = False
                count = 1
                break
        if "Requer" not in address:
            adresses.append({'address': address, 'topic': topic})
        address = ""
    return adresses

In [20]:
#from bson.objectid import ObjectId

#document = collection.find_one(
#        {'_id': ObjectId('5dbc3d2cfcaa0c48eadcac4a')})

### total de pautas que falam de infraestrutura

In [21]:
documents = list(collection.find())
print('%s documentos' % len(list(documents)))
addrs = []
total_pautas = 0
for document in documents: 
    if not document: continue 
    addrs.extend(search_topics_with_address(document))
    total_pautas += len(document.get('pautas', []))
addrs[:1]
print('pautas infra estrutura %s | total de pautas %s' % (len(addrs), total_pautas))
print('%0.2f' % ((len(addrs)*100)/total_pautas))

56 documentos
pautas infra estrutura 1080 | total de pautas 1773
60.91


In [24]:
# import json 
# print(json.dumps(addrs, ensure_ascii=False)) 

### pegar a lat,lng dos endereços

```
import googlemaps


gmaps_client = googlemaps.Client(key=API_KEY_MAPS)
geocode_result = gmaps_client.geocode('Natal, RN') 

locations = []
locations_details = []
for addr in addrs:
        # Geocoding an address
        geocode_result = gmaps_client.geocode(
            '{}, Natal, RN'.format(addr['address']))
        #print('{}, Natal, RN: {}'.format(location['address'], location['topic'])) 
        locations.append((geocode_result[0]['geometry']['location']['lat'], 
                     geocode_result[0]['geometry']['location']['lng']))
        
        details = {'address': addr['address'], 'topic': addr['topic']}
        locations_details.append(details)
```

### salvar em arquivo

```
with open('locations.txt', 'w') as fp:
    fp.write(str(locations))
    
with open('locations_details.txt', 'w') as fp:
    fp.write(str(locations_details))
```

In [22]:
import ast


t = open('locations.txt').read()
locations = list(ast.literal_eval(t))
print(locations[:4])

t = open('locations_details.txt').read()
locations_details = ast.literal_eval(t)
print(locations_details[:2])

[(-5.8750126, -35.2096997), (-5.8646069, -35.2090234), (-5.870620499999999, -35.2082005), (-5.83157, -35.2078552)]
[{'address': 'Rua Araxá Conjunto Pirangi ', 'topic': 'requer que seja enviado oficio a URBANA , solicitando a limpeza do canteiro central da Rua Araxá Conjunto Pirangi I etapa Neópolis'}, {'address': 'Avenida das Alagoas Conjunto ', 'topic': 'Requer que seja enviado ofício à SEMSUR , solicitando a limpeza da Praça Tarcísio Maia , situada na Avenida das Alagoas Conjunto Pirangi I Neópolis'}]


In [23]:
import gmaps
gmaps.configure(api_key=API_KEY_MAPS)

fig = gmaps.figure(map_type='SATELLITE')

heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
import gmaps
gmaps.configure(api_key=API_KEY_MAPS)
 
info_box_template = """
    <dl>
        <dt>Pauta</dt>
        <dd>{topic}</dd> 
    </dl>
"""
locations_info = [info_box_template.format(**location) for location in locations_details]

marker_layer = gmaps.marker_layer(locations, info_box_content=locations_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
info_box_template = """
    <dl>
        <dt>Pauta</dt>
        <dd>{topic}</dd> 
    </dl>
"""
plant_info = [info_box_template.format(**plant) for plant in locations_details]

marker_layer = gmaps.marker_layer(locations, info_box_content=plant_info)
fig = gmaps.figure(layout={ 
        'height': '1200px',  
})
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='1200px'))

In [26]:
# !wget https://gist.githubusercontent.com/gabicavalcante/55698106a7dace94fc2d006f077178ef/raw/a3f28da0ed88abde88db6ebe803272422d1c05d4/favelas-natalrn.txt

In [27]:
# 'Barro Duro'
favelas = []
filepath = 'favelas-natalrn.txt'
with open(filepath) as fp:
    for cnt, line in enumerate(fp):  
        favelas.append(str(line).replace('\n', ''))

print(favelas)

['Vila de Ponta Negra', 'África', 'Alemão', 'Aliança', 'Alta Tensão', 'Alto do Guarapes', 'Aparecida', 'Areado', 'Barreiros', 'Beira Rio', 'Boa Soarte', 'Brasília Teimosa', 'Camboim', 'Coqueiros', 'Cruzeiro', 'Curtume', 'Fio', 'Formigueiro', 'Gramoré', 'José Sarney', 'Lagoinha', 'Lavadeiras', 'Mãe Luíza', 'Salgadinho', 'Maruim', 'Mereto', 'Mosquito', 'Novo Horizonte', 'Ocidental de Baixo', 'Ocidenteal de Cima', 'Passo da Pátria', 'Planalto', 'Pompeia', 'Raio de Sol', 'São José do Jacó', 'Sopapo', 'Tenente Procópio', 'Treze de Maio', 'Vietnã']


In [28]:
favelas_locations = []
favelas_locations_details = []

import googlemaps

gmaps_client = googlemaps.Client(key=API_KEY_MAPS)
geocode_result = gmaps_client.geocode('Natal, RN') 

for favela in favelas:
    # Geocoding an address
    geocode_result = gmaps_client.geocode(
        '{}, Natal, RN'.format(favela)) 
    favelas_locations.append((geocode_result[0]['geometry']['location']['lat'], 
                  geocode_result[0]['geometry']['location']['lng']))

In [29]:
assuntos_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 150, 0, 0.4)', scale=2
)

favelas_layer = gmaps.symbol_layer(
    favelas_locations, fill_color='rgba(200, 0, 0, 0.4)',
    stroke_color='rgba(200, 0, 0, 0.4)', scale=2
)

fig = gmaps.figure(layout={ 
        'height': '1000px',  
})
fig.add_layer(assuntos_layer)
fig.add_layer(favelas_layer)
fig

Figure(layout=FigureLayout(height='1000px'))

In [121]:
#tps://www.brechando.com/2016/04/existe-favela-em-natal/
#http://www.tribunadonorte.com.br/noticia/favela-do-mosquito-estado-de-sa-tio/419728